# Polish Keyboard Layout Optimizer

## Assumptions and goal
Let's dive straight in: our mission is to craft the ultimate keyboard layout for typing in Polish. How do we define "ultimate"? Simple. It's all about minimizing the distance your fingers need to travel while typing. Less travel, more speed, better efficiency.

### What does the best mean?
In the realm of keyboard layouts, "best" is all about efficiency. It's a game of finding a layout where your fingers barely have to move to hit the right keys. Think of it as choreographing a finger ballet—smooth, swift, and efficient.

### Our Target User
Our muse is the classic 10-finger typist, using the traditional home row keys. We're designing for the typist who's all about A, S, D, F, and J, K, L, ; - but with a Polish twist. This means embracing all the unique characters of the Polish language.

### A Note on Variability
Here's where it gets spicy: different datasets lead to different layouts. From formal documents to casual tweets, the variety in language use means our optimal layout might shift a bit. And remember, keyboards themselves are diverse creatures. The spacing and placement of keys can vary, so our ideal layout might not be universally perfect.

## Methodology
It's all about the data. We're diving deep into the CC100-Polish dataset to explore common words and typing patterns, including those unique Polish characters. Feel adventurous? Mix in other datasets for a broader view. Our goal? A keyboard layout that feels like it's reading your mind, with the least amount of finger travel possible.

In [10]:
def calculate_key_distance(key1, key2, layout):
    """
    Calculate the distance between two keys on a given keyboard layout, considering finger positions.
    
    :param key1: First key character
    :param key2: Second key character
    :param layout: Dictionary representing the keyboard layout with positions
    :return: Distance between the two keys
    """
    # Standard distances
    same_row = 1.0
    middle_upper = 1.032
    middle_lower = 1.118
    bottom_top = 2.138

    # Special cases
    special_cases = {
        ('F', 'T'): 1.247, ('H', 'U'): 1.247,
        ('G', 'R'): 1.605, ('J', 'Y'): 1.605,
        ('B', 'F'): 1.803, ('M', 'H'): 1.803,
        ('B', 'R'): 2.661, ('M', 'Y'): 2.661,
        ('V', 'T'): 2.015, ('N', 'U'): 2.015
    }

    # Default finger positions (row, column)
    default_positions = {
        'left_pinky': (1, 0),  # A
        'left_ring': (1, 1),   # S
        'left_middle': (1, 2), # D
        'left_index': (1, 3),  # F
        'right_index': (1, 4), # J
        'right_middle': (1, 5),# K
        'right_ring': (1, 6),  # L
        'right_pinky': (1, 7)  # ;
    }

    # Assign fingers to keys
    finger_assignment = {
        'A': 'left_pinky', 'S': 'left_ring', 'D': 'left_middle', 'F': 'left_index',
        'J': 'right_index', 'K': 'right_middle', 'L': 'right_ring', ';': 'right_pinky',
        # ... other keys ...
    }

    # Check for special case
    if (key1, key2) in special_cases:
        return special_cases[(key1, key2)]
    elif (key2, key1) in special_cases:
        return special_cases[(key2, key1)]

    # Get positions of the keys
    pos1 = layout.get(key1)
    pos2 = layout.get(key2)

    # Determine the fingers used for each key
    finger1 = finger_assignment.get(key1)
    finger2 = finger_assignment.get(key2)

    # Calculate distance
    if pos1 and pos2 and finger1 and finger2:
        # If different fingers are used, calculate from the default position of the second finger
        if finger1 != finger2:
            default_pos2 = default_positions.get(finger2)
            row_diff = abs(default_pos2[0] - pos2[0])
            col_diff = abs(default_pos2[1] - pos2[1])
        else:  # If the same finger is used, calculate directly between the keys
            row_diff = abs(pos1[0] - pos2[0])
            col_diff = abs(pos1[1] - pos2[1])

        # Same row
        if row_diff == 0:
            return col_diff * same_row
        # Adjacent rows (middle to upper or lower)
        elif row_diff == 1:
            return (col_diff * same_row) + (middle_upper if (1 in [default_pos2[0], pos2[0]]) else middle_lower)
        # Bottom to top
        elif row_diff == 2:
            return (col_diff * same_row) + bottom_top
    return 0  # Return 0 if keys are not found or the same

# Test the function with an example
calculate_key_distance('A', 'Q', example_layout)  # Example with different fingers (A to R)

0

# 